<a href="https://colab.research.google.com/github/davidwhogg/WorseThanLSP/blob/main/notebooks/fit_frequencies_to_irregular_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fit K frequencies to N data points, but fast
Making use of the fastness of `finufft`.

## Author:
- **David W. Hogg** *(NYU) (MPIA) (Flatiron)*

## Notes:
- This notebook defines `finufft` and `traditional` fitting routines. On test cases with small numbers of frequencies, the traditional methods will be faster. It remains to be seen whether, for large numbers of frequencies, the `finufft` routines are way faster.

## Bugs:
- Needs to move to GitHub from here.
- Barely tested.
- None of the code currently makes use of individual data-point uncertainty variances.

## To-do items:
- Do a full comparison to building a design matrix and fitting as per usual.

In [ ]:
!pip install finufft

In [ ]:
# essential imports
import numpy as np
import scipy.sparse.linalg as sp
from functools import partial
from scipy.fftpack import fftfreq
import finufft

In [ ]:
# non-essential imports
import matplotlib.pyplot as plt

In [ ]:
# This set of functions does the traditional fitting Hogg loves

def _hogg_design_matrix(fs, ts):
    assert np.all(fs > 0.)
    N = len(ts)
    X = np.ones_like(ts)
    for f in fs:
        X = np.vstack([np.exp(-2.j * np.pi * f * ts), X, np.exp(2.j * np.pi * f * ts)])
    return X.T

def hogg_traditional_fit(ts, ys, fs):
    X = _hogg_design_matrix(fs, ts)
    return np.linalg.solve(X.T @ X, X.T @ ys)

def hogg_traditional_synthesize(fs, Zs, ts):
    return _hogg_design_matrix(fs, ts) @ Zs

In [ ]:
# This block of code defines a pure pseudo-inverse of the nufft1d3() function.

FEPS, ATOL, BTOL = 1.e-6, 1.e-6, 1.e-6 # made up

def nufft1d3_pinv(x, c, s):
    """
    The pseudo-inverse of `nufft1d3()`.

    ## Notes:

    ## Bugs:
    - Based on experimental coding.
    """
    M = len(x)
    N = len(s)
    R = lambda f: finufft.nufft1d3(s, f, x, eps=FEPS)
    RT = lambda c: finufft.nufft1d3(x, c, s, eps=FEPS, isign=-1)
    f0 = RT(c)
    RR = sp.LinearOperator((M, N), matvec=R, rmatvec=RT, dtype=np.complex128)
    res = sp.lsqr(RR, c, x0=f0, atol=ATOL, btol=BTOL)
    print("nufft1d3_pinv(): completed in", res[2], "iterations")
    return res[0]

In [ ]:
# This set of functions wraps the finufft pseudo-inverse into a more useful
# fitting code.

def _hogg_delta_omega(ts):
    """
    ## Bugs:
    - Doesn't have a proper code header.
    """
    Nt = len(ts)
    assert ts.shape == (Nt, )
    return 2. * np.pi * (Nt / (Nt + 1)) / (max(ts) - min(ts))

def _hogg_make_frequency_list(fs, Delta_omega):
    """
    ## Bugs:
    - Doesn't have a proper code header.
    """
    assert np.all(fs > 0.)
    return 2. * np.pi * np.concatenate([-1. * np.flip(fs), [0., ], fs]) / Delta_omega

def hogg_finufft_fit(ts, ys, fs, Delta_omega=None, t_offset=None):
    """
    ## Inputs:
    - `ts`: the times of the `N` points
    - `ys`: the values at the `N` times
    - `fs`: the `K` frequencies in play; all should be strictly positive
    - `Delta_omega`: use with care
    - `t_offset`: use with care

    ## Bugs / issues:
    - Don't mess with `Delta_omega` and `offset` unless under good supervision.
    - May barf if some frequencies get very high??
    - This makes decisions "for" the user.
    - Output Zs have really weird units, people.
    - Doesn't have a proper code header.
    """
    N = len(ts)
    assert ts.shape == ys.shape
    # choose a sensible conversion of `ts` to dimensionless positions `xs`.
    if t_offset is None:
        t_offset = np.nanmedian(ts)
    if Delta_omega is None:
        Delta_omega = _hogg_delta_omega(ts)
    xs = Delta_omega * (ts - t_offset)
    assert (max(xs) - min(xs)) < 2. * np.pi
    # convert input frequencies `fs` to dimensionless frequencies (signals) `ss`.
    ss = _hogg_make_frequency_list(fs, Delta_omega)
    # run `finufft` pseudo-inverse.
    Zs = nufft1d3_pinv(xs, np.complex128(ys), ss)
    # convert output to frequency units.
    return Zs

def hogg_finufft_synthesize(fs, Zs, ts, Delta_omega=None, t_offset=None):
    """
    ## Inputs:
    - `fs`: frequencies; must be strictly positive
    - `Zs`: complex amplitudes
    - `ts`: time locations of points
    - `Delta_omega`:
    - `t_offset`:

    ## Bugs:
    - Don't mess with `Delta_omega` and `t_offset` unless under good supervision.
    - Incomplete comment header.
    """
    if Delta_omega is None:
        Delta_omega = _hogg_delta_omega(ts)
    if t_offset is None:
        t_offset = np.nanmedian(ts)
    xs = Delta_omega * (ts - t_offset)
    ss = _hogg_make_frequency_list(fs, Delta_omega)
    assert ss.shape == Zs.shape
    return finufft.nufft1d3(ss, Zs, xs)

In [ ]:
# make some fake data
rng = np.random.default_rng(17)
ts = np.sort(1.6 * rng.uniform(size=512))
truefs = np.array([1.5, 3.99, 11.])
trueZs = np.array([0.06, 0.03, 0.01])
ys = 1. + 0.0 * ts
for Z,f in zip(trueZs, truefs):
    ys += Z * np.cos(2. * np.pi * f * ts)
ys += 0.01 * np.random.normal(size=len(ys))
plt.plot(ts, ys, "k.")

In [ ]:
Zs = hogg_finufft_fit(ts, ys, truefs)
print(Zs)

In [ ]:
# This reconstructs the synthesized data from the fit output.
synth_ys = hogg_finufft_synthesize(truefs, Zs, ts)
print(synth_ys.shape)

In [ ]:
# This makes more synthesized data from the fit and plots it.
T = np.max(ts) - np.min(ts)
plot_ts = np.arange(np.min(ts) - 0.05 * T, np.max(ts) + 0.05 * T, 0.001 * T)
plot_ys = hogg_finufft_synthesize(truefs, Zs, plot_ts,
                               Delta_omega=_hogg_delta_omega(ts),
                               t_offset=np.nanmedian(ts))
plt.plot(plot_ts, plot_ys, "r-")
plt.plot(ts, ys, "k.")

In [ ]:
tZs = hogg_traditional_fit(ts, ys, truefs)
print(tZs)

In [ ]:
# This makes more synthesized data from the fit and plots it.
tplot_ys = hogg_traditional_synthesize(truefs, tZs, plot_ts)
plt.plot(plot_ts, tplot_ys, "r-")
plt.plot(ts, ys, "k.")